Neural Networks


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Read the CSV file into a pandas DataFrame
data = pd.read_csv('C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Data/merged_train_sameday_real.csv')
test_data = pd.read_csv('C:/Users/maxva/OneDrive - Tilburg University/Msc. Data Science/Master Thesis/Data/merged_test_sameday_real.csv')
columns_to_remove = ['volume', 'open_interest', 'option_price']
test_data = test_data.drop(columns=columns_to_remove)
data = data.drop(columns=columns_to_remove)
test_data_with_all_predictions = test_data.copy() 

option_columns = [
    'impl_volatility',
    'cp_flag',
    'stock_price',
    'moneyness',
    'time_to_expiry',
    'strike_price',
    'delta',
    'gamma',
    'vega',
    'theta',
    'rf',
    'iv_ahbs',
    'iv_ahbs_error',
    'iv_bs',
    'iv_bs_error',
    'iv_cw',
    'iv_cw_error'
]

option_only = data[option_columns]
print(option_only)

         impl_volatility  cp_flag  stock_price  moneyness  time_to_expiry  \
0               0.210270        1     115.5450   0.970966              24   
1               0.208124        1     115.5450   0.962875              24   
2               0.205474        1     115.5450   0.954917              24   
3               0.278442        0     115.5450   1.100429              24   
4               0.242212        0     115.5450   1.050409              24   
...                  ...      ...          ...        ...             ...   
1634738         0.231520        1     111.0158   0.965355             141   
1634739         0.259238        0     111.0158   1.110158             141   
1634740         0.247787        0     111.0158   1.057293             141   
1634741         0.236913        0     111.0158   1.009235             141   
1634742         0.265734        0     111.0158   1.138624             141   

         strike_price     delta     gamma       vega      theta      rf  \


In [ ]:
###########################################
# PART 1: LIGHTGBM MODEL DEFINITION
###########################################

def create_lgb_model(model_type):

    if model_type == 'LGB1':
        # Standard LightGBM configuration
        params = {
            'objective': 'regression',
            'metric': 'mse',
            'boosting_type': 'gbdt',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': -1
        }
    
    elif model_type == 'LGB2':
        # More complex LightGBM configuration with different hyperparameters
        params = {
            'objective': 'regression',
            'metric': 'mse',
            'boosting_type': 'gbdt',
            'num_leaves': 63,
            'learning_rate': 0.01,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.7,
            'bagging_freq': 4,
            'min_data_in_leaf': 20,
            'max_depth': 10,
            'verbose': -1
        }
    
    else:
        raise ValueError("Invalid model type. Choose from 'LGB1' or 'LGB2'.")
    
    return params

def train_and_evaluate_model(params, X_train, y_train, X_test, y_test, num_boost_round=100):
    # Create LightGBM datasets
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    # Train model with early stopping
    model = lgb.train(
        params,
        train_data,
        num_boost_round=num_boost_round,
        valid_sets=[valid_data],
        callbacks=[lgb.early_stopping(stopping_rounds=20, verbose=True)]
    )
    
    # Evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    
    return model, mse

In [ ]:
###########################################
# PART 2: DATA PREPARATION
###########################################

def prepare_data(option_only):

    # Prepare features and target variables
    feature_columns = [col for col in option_only.columns if col not in 
                      ['iv_bs_error', 'iv_ahbs', "iv_ahbs_error", "iv_bs", 
                       "iv_cw", "iv_cw_error", "impl_volatility"]]
    
    # Instead of using train_test_split, use your predefined sets
    X_train = option_only[feature_columns]  # Features from your training set
    X_test = test_data[feature_columns]  # Features from your test set

    # Target variables for each error type
    y_bs_train = option_only['iv_bs_error']  
    y_bs_test = test_data['iv_bs_error']  

    y_ahbs_train = option_only['iv_ahbs_error']  
    y_ahbs_test = test_data['iv_ahbs_error'] 

    y_cw_train = option_only['iv_cw_error']  
    y_cw_test = test_data['iv_cw_error'] 
    
    # Initialize a StandardScaler to normalize the features
    scaler = StandardScaler()
    
    # Fit the scaler on the training data and transform both training and testing data
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Create dictionaries to store multiple target variables
    y_train_dict = {
        'bs': y_bs_train,
        'ahbs': y_ahbs_train,
        'cw': y_cw_train
    }
    
    y_test_dict = {
        'bs': y_bs_test,
        'ahbs': y_ahbs_test,
        'cw': y_cw_test
    }
    
    return X_train_scaled, X_test_scaled, y_train_dict, y_test_dict, scaler, feature_columns


In [ ]:
###########################################
# PART 3: PREDICTION FUNCTION
###########################################

def predict_and_add_to_test_data(models, test_data, scaler, feature_columns, error_type='bs'):

    # Create a copy of the test data to avoid modifying the original
    result_df = test_data.copy()
    
    # Extract features from test data
    X_test = test_data[feature_columns].values
    
    # Scale the features using the pre-fitted scaler
    X_test_scaled = scaler.transform(X_test)
    
    # Original value column name
    original_column = f'iv_{error_type}'
    
    # Generate predictions for each model
    for model_name, model in models.items():
        # Make predictions
        predictions = model.predict(X_test_scaled)
        
        # Add predictions to the dataframe
        column_name = f'iv_{error_type}_pred_{model_name}'
        result_df[column_name] = predictions
        
        # Calculate corrected value by adding the error prediction to the original value
        result_df[f'iv_{error_type}_corrected_{model_name}'] = result_df[original_column] - predictions
    
    return result_df

In [5]:
###########################################
# PART 4: FEATURE IMPORTANCE ANALYSIS
###########################################

def analyze_feature_importance(model, feature_columns):
    """
    Extract and display feature importance from a LightGBM model.
    
    Parameters:
    model: Trained LightGBM model
    feature_columns (list): List of feature column names
    
    Returns:
    pandas.DataFrame: DataFrame with feature importance scores
    """
    # Get feature importance
    importance = model.feature_importance(importance_type='split')
    
    # Create a DataFrame for better visualization
    importance_df = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': importance
    })
    
    # Sort by importance
    importance_df = importance_df.sort_values('Importance', ascending=False)
    
    return importance_df


In [6]:

###########################################
# PART 5: COMPLETE WORKFLOW
###########################################

if __name__ == "__main__":
    # Step 1: Prepare data
    X_train_scaled, X_test_scaled, y_train_dict, y_test_dict, scaler, feature_columns = prepare_data(option_only)
    
    # Step 2: Train models for each target variable
    models = {}
    results = {}
    
    # Dictionary to store all models
    all_models = {
        'bs': {},
        'ahbs': {},
        'cw': {}
    }
    
    # Train models for each error type
    for error_type in ['bs', 'ahbs', 'cw']:
        print(f"\n=== Training models for {error_type} error correction ===")
        
        # Get the appropriate training and test targets
        y_train = y_train_dict[error_type]
        y_test = y_test_dict[error_type]
        
        # Train each model configuration
        for lgb_type in ['LGB1', 'LGB2']:
            model_name = f"{lgb_type}_{error_type}"
            print(f"\nTraining {model_name}...")
            
            # Create model parameters
            params = create_lgb_model(lgb_type)
            
            # Train and evaluate model
            model, mse = train_and_evaluate_model(
                params, X_train_scaled, y_train, X_test_scaled, y_test, num_boost_round=500
            )
            
            # Store model and results
            all_models[error_type][lgb_type] = model
            results[model_name] = {
                'mse': mse,
                'feature_importance': analyze_feature_importance(model, feature_columns)
            }
            
            print(f"{model_name} Test MSE: {mse}")
            
            # Save feature importance
            results[model_name]['feature_importance'].to_csv(f"{model_name}_feature_importance_sameday.csv", index=False)
            
            # Save model (using LightGBM's save_model method)
            model.save_model(f"{model_name}_model_sameday.txt")
    
    # Step 3: Generate predictions on test data for each error type
    for error_type in ['bs', 'ahbs', 'cw']:
        print(f"\n=== Making predictions for {error_type} error correction ===")
        
        # Extract the models for this error type
        current_models = all_models[error_type]
        
        # Make predictions - use the accumulated DataFrame
        test_data_with_all_predictions = predict_and_add_to_test_data(
            current_models, test_data_with_all_predictions, scaler, feature_columns, error_type
        )
        
        # Save intermediate results if desired
        test_data_with_all_predictions.to_csv(f'test_data_with_{error_type}_predictions_sameday.csv', index=False)
        
        # Display sample results from accumulated DataFrame
        print(f"\nSample of test data with {error_type} predictions:")
        display_columns = ['iv_' + error_type]
        for lgb_type in ['LGB1', 'LGB2']:
            display_columns.extend([
                f'iv_{error_type}_pred_{lgb_type}', 
                f'iv_{error_type}_corrected_{lgb_type}'
            ])
        
        print(test_data_with_all_predictions[display_columns].head(5))

    # Final save with all predictions
    test_data_with_all_predictions.to_csv('test_data_with_all_predictions_sameday.csv', index=False)


=== Training models for bs error correction ===

Training LGB1_bs...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.000226269
LGB1_bs Test MSE: 0.0002262685848857054

Training LGB2_bs...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.000609212
LGB2_bs Test MSE: 0.0006092122491386029

=== Training models for ahbs error correction ===

Training LGB1_ahbs...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.000223554
LGB1_ahbs Test MSE: 0.00022355414000430285

Training LGB2_ahbs...
Training until validation scores don't improve for 20 rounds
Did not meet early stopping. Best iteration is:
[500]	valid_0's l2: 0.000587698
LGB2_ahbs Test MSE: 0.0005876983208977582

=== Training models for cw error correction ===

Training LGB1_cw...
Training unti

C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Sample of test data with bs predictions:
      iv_bs  iv_bs_pred_LGB1  iv_bs_corrected_LGB1  iv_bs_pred_LGB2  \
0  0.353798         0.152401              0.201398         0.147649   
1  0.353798         0.149811              0.203987         0.149893   
2  0.353798         0.152491              0.201307         0.152401   
3  0.353798         0.148667              0.205131         0.148020   
4  0.353798         0.127547              0.226251         0.121123   

   iv_bs_corrected_LGB2  
0              0.206149  
1              0.203905  
2              0.201397  
3              0.205779  
4              0.232675  

=== Making predictions for ahbs error correction ===


C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Sample of test data with ahbs predictions:
    iv_ahbs  iv_ahbs_pred_LGB1  iv_ahbs_corrected_LGB1  iv_ahbs_pred_LGB2  \
0  0.350970           0.150908                0.200063           0.149187   
1  0.349422           0.149656                0.199765           0.152786   
2  0.348100           0.149932                0.198168           0.154409   
3  0.344389           0.137132                0.207257           0.138896   
4  0.359680           0.135478                0.224202           0.132251   

   iv_ahbs_corrected_LGB2  
0                0.201783  
1                0.196636  
2                0.193691  
3                0.205492  
4                0.227429  

=== Making predictions for cw error correction ===


C:\Users\maxva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



Sample of test data with cw predictions:
      iv_cw  iv_cw_pred_LGB1  iv_cw_corrected_LGB1  iv_cw_pred_LGB2  \
0  0.316475         0.118035              0.198440         0.113300   
1  0.315673         0.114406              0.201267         0.116041   
2  0.315839         0.118137              0.197703         0.117249   
3  0.341157         0.133201              0.207955         0.133796   
4  0.329592         0.104031              0.225561         0.099750   

   iv_cw_corrected_LGB2  
0              0.203175  
1              0.199632  
2              0.198590  
3              0.207361  
4              0.229842  


Calculate IVRMSE

In [ ]:

def calculate_ivrmse(predictions_df, error_types=['bs', 'ahbs', 'cw'], models=['LGB1', 'LGB2']):

    results = {}
    
    # Calculate IVRMSE for each error type and model
    for error_type in error_types:
        orig_col = f'iv_{error_type}'
        
        # Calculate base IVRMSE (before correction)
        base_rmse = np.sqrt(mean_squared_error(predictions_df['impl_volatility'], predictions_df[orig_col]))
        results[f"{error_type}_base"] = base_rmse
        
        # Calculate IVRMSE for each model
        for model in models:
            corrected_col = f'iv_{error_type}_corrected_{model}'
            
            # Skip if corrected column doesn't exist
            if corrected_col not in predictions_df.columns:
                print(f"Warning: {corrected_col} column not found, skipping...")
                continue
                
            # Calculate IVRMSE for the corrected predictions
            corrected_rmse = np.sqrt(mean_squared_error(predictions_df['impl_volatility'], predictions_df[corrected_col]))
            results[f"{error_type}_{model}"] = corrected_rmse
            
            # Calculate improvement percentage
            improvement = (base_rmse - corrected_rmse) / base_rmse * 100
            results[f"{error_type}_{model}_improvement"] = improvement
    
    return results

# Example usage
if __name__ == "__main__":
    # Load the predictions DataFrame (assuming it was saved in the main script)
    test_data_with_all_predictions = pd.read_csv('test_data_with_all_predictions_sameday.csv')
    
    # Calculate IVRMSE
    ivrmse_results = calculate_ivrmse(test_data_with_all_predictions)
    
    # Print results in a table format
    print("\n=== IVRMSE Results ===")
    print(f"{'Model':<15} {'IVRMSE':<10} {'Improvement':<12}")
    print("-" * 40)
    
    for error_type in ['bs', 'ahbs', 'cw']:
        base_key = f"{error_type}_base"
        if base_key in ivrmse_results:
            base_rmse = ivrmse_results[base_key]
            print(f"{error_type.upper():<15} {base_rmse:.6f}  {'(baseline)':<12}")
            
            for model in ['LGB1', 'LGB2']:
                model_key = f"{error_type}_{model}"
                imp_key = f"{error_type}_{model}_improvement"
                
                if model_key in ivrmse_results:
                    print(f"{model_key:<15} {ivrmse_results[model_key]:.6f}  {ivrmse_results[imp_key]:.2f}%")
            print("-" * 40)
    
    # Find best overall model
    model_keys = [k for k in ivrmse_results.keys() if not k.endswith('base') and not k.endswith('improvement')]
    if model_keys:
        best_model = min(model_keys, key=lambda k: ivrmse_results[k])
        print(f"\nBest overall model: {best_model} with IVRMSE = {ivrmse_results[best_model]:.6f}")
        
        base_key = f"{best_model.split('_')[0]}_base"
        imp_key = f"{best_model}_improvement"
        if base_key in ivrmse_results and imp_key in ivrmse_results:
            print(f"Improvement over baseline: {ivrmse_results[imp_key]:.2f}%")
            
    # Export results to CSV for further analysis
    results_df = pd.DataFrame({
        'Model': list(ivrmse_results.keys()),
        'Value': list(ivrmse_results.values())
    })
    results_df.to_csv('ivrmse_results_lightgbm_sameday.csv', index=False)


=== IVRMSE Results ===
Model           IVRMSE     Improvement 
----------------------------------------
BS              0.150568  (baseline)  
bs_LGB1         0.015042  90.01%
bs_LGB2         0.024682  83.61%
----------------------------------------
AHBS            0.134068  (baseline)  
ahbs_LGB1       0.014952  88.85%
ahbs_LGB2       0.024242  81.92%
----------------------------------------
CW              0.128764  (baseline)  
cw_LGB1         0.015881  87.67%
cw_LGB2         0.024510  80.97%
----------------------------------------

Best overall model: ahbs_LGB1 with IVRMSE = 0.014952
Improvement over baseline: 88.85%


In [ ]:

# Define the moneyness groups
moneyness_groups = ['DOTMC', 'OTMC', 'ATM', 'OTMP', 'DOTMP']

def analyze_ivrmse_by_moneyness(test_data):

    # 1. Group the test data by moneyness
    results = {}
    for group in moneyness_groups:
        group_data = test_data[test_data['moneyness_category'] == group]
        
        # Skip if no data in this group
        if len(group_data) == 0:
            print(f"Warning: No data found for moneyness group {group}")
            continue
            
        # Initialize results for this group
        results[group] = {}
        
        # 2. Calculate IVRMSE for each error type and model
        for error_type in ['bs', 'ahbs', 'cw']:
            # Column names
            orig_col = f'iv_{error_type}'
            
            # Skip if column doesn't exist
            if orig_col not in group_data.columns:
                print(f"Warning: Column {orig_col} not found, skipping...")
                continue
                
            # Calculate baseline IVRMSE
            base_rmse = np.sqrt(mean_squared_error(
                group_data['impl_volatility'], 
                group_data[orig_col]
            ))
            
            # Store baseline IVRMSE
            results[group][f"{error_type}_base"] = base_rmse
            
            # Calculate corrected IVRMSE for each model
            for model in ['LGB1', 'LGB2']:
                corrected_col = f'iv_{error_type}_corrected_{model}'
                
                # Skip if column doesn't exist
                if corrected_col not in group_data.columns:
                    print(f"Warning: Column {corrected_col} not found, skipping...")
                    continue
                    
                corrected_rmse = np.sqrt(mean_squared_error(
                    group_data['impl_volatility'], 
                    group_data[corrected_col]
                ))
                
                # Calculate improvement percentage
                improvement = (base_rmse - corrected_rmse) / base_rmse * 100
                
                # Store results
                results[group][f"{error_type}_{model}"] = corrected_rmse
                results[group][f"{error_type}_{model}_improvement"] = improvement
    
    return results

def format_results_table(results):

    # Prepare IVRMSE table data
    ivrmse_data = []
    for group, group_results in results.items():
        row = {'Moneyness Group': group}
        
        for key, value in group_results.items():
            if not key.endswith('improvement'):
                if key.endswith('base'):
                    # Format baseline columns
                    error_type = key.split('_')[0].upper()
                    row[f"{error_type} Base"] = value
                else:
                    # Format model columns
                    parts = key.split('_')
                    error_type = parts[0].upper()
                    model = parts[1]
                    row[f"{error_type} {model}"] = value
        
        ivrmse_data.append(row)
    
    # Prepare improvement percentage table data
    improvement_data = []
    for group, group_results in results.items():
        row = {'Moneyness Group': group}
        
        for key, value in group_results.items():
            if key.endswith('improvement'):
                # Format improvement columns
                parts = key.replace('_improvement', '').split('_')
                error_type = parts[0].upper()
                model = parts[1]
                row[f"{error_type} {model}"] = value
        
        improvement_data.append(row)
    
    # Create DataFrames
    ivrmse_df = pd.DataFrame(ivrmse_data)
    improvement_df = pd.DataFrame(improvement_data)
    
    # Sort columns for better readability
    ivrmse_cols = ['Moneyness Group']
    improvement_cols = ['Moneyness Group']
    
    for et in ['BS', 'AHBS', 'CW']:
        ivrmse_cols.extend([f"{et} Base", f"{et} LGB1", f"{et} LGB2"])
        improvement_cols.extend([f"{et} LGB1", f"{et} LGB2"])
    
    # Reorder columns if they exist
    ivrmse_df = ivrmse_df[[col for col in ivrmse_cols if col in ivrmse_df.columns]]
    improvement_df = improvement_df[[col for col in improvement_cols if col in improvement_df.columns]]
    
    return ivrmse_df, improvement_df

def find_best_models(results):

    best_models = []
    
    for group, group_results in results.items():
        for error_type in ['bs', 'ahbs', 'cw']:
            # Get baseline IVRMSE
            base_key = f"{error_type}_base"
            if base_key not in group_results:
                continue
                
            base_rmse = group_results[base_key]
            
            # Find best model for this error type
            models = [m for m in ['LGB1', 'LGB2'] if f"{error_type}_{m}" in group_results]
            if not models:
                continue
            
            # Find model with lowest IVRMSE    
            best_model = min(models, key=lambda m: group_results[f"{error_type}_{m}"])
            best_rmse = group_results[f"{error_type}_{best_model}"]
            improvement = group_results[f"{error_type}_{best_model}_improvement"]
            
            # Also get the values for the other model for comparison
            other_models = [m for m in models if m != best_model]
            other_model_data = {}
            if other_models:
                other_model = other_models[0]
                other_rmse = group_results[f"{error_type}_{other_model}"]
                other_improvement = group_results[f"{error_type}_{other_model}_improvement"]
                other_model_data = {
                    f'Other Model': other_model,
                    f'Other IVRMSE': other_rmse,
                    f'Other Improvement %': other_improvement
                }
            
            model_data = {
                'Moneyness Group': group,
                'Error Type': error_type.upper(),
                'Best Model': best_model,
                'Base IVRMSE': base_rmse,
                'Best IVRMSE': best_rmse,
                'Improvement %': improvement
            }
            
            # Add other model data if available
            model_data.update(other_model_data)
            
            best_models.append(model_data)
    
    return pd.DataFrame(best_models)

def print_formatted_tables(results):

    # Format results into DataFrames
    ivrmse_df, improvement_df = format_results_table(results)
    
    # Format IVRMSE table
    pd.set_option('display.float_format', '{:.6f}'.format)
    print("=" * 80)
    print("IVRMSE by Moneyness Group (Sameday)")
    print("=" * 80)
    print(ivrmse_df.to_string(index=False))
    
    # Format improvement table
    pd.set_option('display.float_format', '{:.2f}%'.format)
    print("\n" + "=" * 80)
    print("Improvement Percentage by Moneyness Group")
    print("=" * 80)
    print(improvement_df.to_string(index=False))
    
    # Find best models
    best_models_df = find_best_models(results)
    
    # Reset float format for mixed table
    pd.set_option('display.float_format', None)
    print("\n" + "=" * 80)
    print("Best Model by Moneyness Group and Error Type")
    print("=" * 80)
    # Format specific columns
    best_models_df['Base IVRMSE'] = best_models_df['Base IVRMSE'].map('{:.6f}'.format)
    best_models_df['Best IVRMSE'] = best_models_df['Best IVRMSE'].map('{:.6f}'.format)
    best_models_df['Improvement %'] = best_models_df['Improvement %'].map('{:.2f}%'.format)
    print(best_models_df.to_string(index=False))
    
    # Summary statistics
    print("\n" + "=" * 80)
    print("Summary Statistics")
    print("=" * 80)
    
    # Count best model occurrences
    model_counts = best_models_df['Best Model'].value_counts()
    print(f"Overall best model distribution: {dict(model_counts)}")
    
    # Average improvement by moneyness group
    print("\nAverage improvement by moneyness group:")
    # Convert percentage strings to numeric values
    best_models_df['Improvement_Numeric'] = pd.to_numeric(best_models_df['Improvement %'].str.rstrip('%'))
    
    # Group and calculate means
    avg_improvement = best_models_df.groupby('Moneyness Group')['Improvement_Numeric'].mean()
    
    # Handle the case where there's only one group (which returns a scalar)
    if isinstance(avg_improvement, pd.Series):
        # Sort if it's a Series with multiple values
        sorted_improvements = avg_improvement.sort_values(ascending=False)
        for group, imp in sorted_improvements.items():
            print(f"  {group}: {imp:.2f}%")
    else:
        # Just print the single value if it's a scalar
        group = best_models_df['Moneyness Group'].iloc[0]
        print(f"  {group}: {avg_improvement:.2f}%")
    
    # Save results to CSV files
    ivrmse_df.to_csv('ivrmse_by_moneyness_lightgbm.csv', index=False)
    improvement_df.to_csv('improvement_by_moneyness_lightgbm.csv', index=False)
    best_models_df.to_csv('best_models_by_moneyness_lightgbm.csv', index=False)

if __name__ == "__main__":
    # Load test data with predictions
    try:
        test_data_with_all_predictions = pd.read_csv('test_data_with_all_predictions_sameday.csv')
        
        # Check if 'moneyness_category' exists, if not, create it
        if 'moneyness_category' not in test_data_with_all_predictions.columns:
            print("Creating moneyness categories...")
            # Define moneyness boundaries
            test_data_with_all_predictions['moneyness_category'] = pd.cut(
                test_data_with_all_predictions['moneyness'],
                bins=[-float('inf'), 0.9, 0.97, 1.03, 1.1, float('inf')],
                labels=moneyness_groups
            )
        
        # Run analysis
        results = analyze_ivrmse_by_moneyness(test_data_with_all_predictions)
        print_formatted_tables(results)
        
    except FileNotFoundError:
        print("Error: Test data file not found. Please run the main script first.")

IVRMSE by Moneyness Group (Sameday)
Moneyness Group  BS Base  BS LGB1  BS LGB2  AHBS Base  AHBS LGB1  AHBS LGB2  CW Base  CW LGB1  CW LGB2
          DOTMC 0.135785 0.017140 0.026631   0.135335   0.016947   0.026068 0.122504 0.018262 0.024891
           OTMC 0.134271 0.012961 0.021242   0.126841   0.013110   0.020663 0.125176 0.014471 0.021329
            ATM 0.140269 0.013238 0.022637   0.135411   0.013368   0.022475 0.131622 0.014085 0.023905
           OTMP 0.131156 0.012939 0.021602   0.129842   0.012750   0.022723 0.128338 0.013326 0.022951
          DOTMP 0.195506 0.018446 0.030369   0.142425   0.018106   0.029034 0.134354 0.018785 0.028941

Improvement Percentage by Moneyness Group
Moneyness Group  BS LGB1  BS LGB2  AHBS LGB1  AHBS LGB2  CW LGB1  CW LGB2
          DOTMC   87.38%   80.39%     87.48%     80.74%   85.09%   79.68%
           OTMC   90.35%   84.18%     89.66%     83.71%   88.44%   82.96%
            ATM   90.56%   83.86%     90.13%     83.40%   89.30%   81.84%
       